<a href="https://colab.research.google.com/github/rudraser/Fluter-BasicApps/blob/main/Custom_ResNet_attempt1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import torch
import torchvision # torch package for vision related things
import torch.nn.functional as F  # Parameterless functions, like (some) activation functions
import torchvision.datasets as datasets  # Standard datasets
import torchvision.transforms as transforms  # Transformations we can perform on our dataset for augmentation
from torch import optim  # For optimizers like SGD, Adam, etc.
from torch import nn  # All neural network modules
from torch.utils.data import DataLoader  # Gives easier dataset managment by creating mini batches etc.
from tqdm import tqdm  # For nice progress bar!
device = (torch.device('cuda') if torch.cuda.is_available()
          else torch.device('cpu'))
print(f"Training on device {device}.")

Training on device cuda.


In [32]:

from matplotlib import pyplot as plt
import numpy as np
import collections


import torch.nn as nn

import torch.optim as optim

from torchvision import datasets, transforms
data_path = '/data-unversioned/p1ch6/'
cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(size=[32,32], padding=4),                        
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

Files already downloaded and verified


In [33]:
from torchvision import datasets, transforms
data_path = '/data-unversioned/p1ch6/'
cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(size=[32,32], padding=4),                        
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

Files already downloaded and verified


In [40]:
cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=True,
    transform=transforms.Compose([               #loading the val set
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

Files already downloaded and verified


In [34]:
class resblock(nn.Module):                    #creating a resblock which would be used iterativealy.
    def __init__(self, n):
        super(resblock, self).__init__()
        self.conv = nn.Conv2d(n, n, kernel_size=3,
                              padding=1, bias=False)                 # defining layers for forward
        self.batch_norm = nn.BatchNorm2d(num_features=n)

        torch.nn.init.kaiming_normal_(self.conv.weight,
                                      nonlinearity='relu')  
        torch.nn.init.constant_(self.batch_norm.weight, 0.5)
        torch.nn.init.zeros_(self.batch_norm.bias)

    def forward(self, x):
        out = self.conv(x)                                 #Actual resblock with the skip connection over 1 layer
        out = self.batch_norm(out)
        out = torch.relu(out)
        return out + x

In [35]:
class deep(nn.Module):                # the model which uses resblock iteratively and some other layers
    def __init__(self, n1=32, n_blocks=10):
        super().__init__()
        self.n1 = n1
        self.conv1 = nn.Conv2d(3, n1, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(n1, 2*n1, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(2*n1, 4*n1, kernel_size=3, padding=1)       # defing layers for forward
        self.resblocks = nn.Sequential(
            *(n_blocks * [resblock(n=n1)]))
        self.resblocks2 = nn.Sequential(
            *((n_blocks*2) * [resblock(n=n1*2)]))
        self.resblocks3 = nn.Sequential(
            *(n_blocks * [resblock(n=n1*4)]))
        self.fc1 = nn.Linear(8 * 8 * n1*4, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 10)
        
    def forward(self, x):                    # using the resblocks 3 times, each with twice the number of channels as before
        out = torch.relu(self.conv1(x))
        out = self.resblocks(out)                          
        out = F.max_pool2d(out, 2) 
        out = self.conv2(out)
        out = self.resblocks2(out)
        out = F.max_pool2d(out, 2) 
        out = self.conv3(out)
        out = self.resblocks3(out)
        out = out.view(-1, 8 * 8 * self.n1*4)
        out = torch.relu(self.fc1(out))
        out = torch.relu(self.fc2(out))
        out = self.fc3(out)
        return out

In [36]:
import datetime

def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):           # defining the training loop
    for epoch in range(1, n_epochs + 1):                                        # these inputs would be defined in next cell
        loss_train = 0.0
        for imgs, labels in train_loader:
            imgs = imgs.to(device=device)  # <1>
            labels = labels.to(device=device)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_train += loss.item()

        
        print('{} Epoch {}, Training loss {}'.format(
            datetime.datetime.now(), epoch,
            loss_train / len(train_loader)))

In [37]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                           shuffle=True)                   # defining mini batch size and shuffling

model =  deep().to(device=device)# <1>
optimizer = optim.SGD(model.parameters(), lr=1e-2, weight_decay=0.01)
loss_fn = nn.CrossEntropyLoss()

In [38]:

training_loop(                   #calling the training loop finally
    n_epochs = 60,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)

2021-07-31 05:00:39.970732 Epoch 1, Training loss 1.7361613687346964
2021-07-31 05:01:46.261139 Epoch 2, Training loss 1.3139591197223615
2021-07-31 05:02:52.369499 Epoch 3, Training loss 1.0961830253948641
2021-07-31 05:03:58.608086 Epoch 4, Training loss 0.9618200964634985
2021-07-31 05:05:04.779977 Epoch 5, Training loss 0.878183582669024
2021-07-31 05:06:11.018292 Epoch 6, Training loss 0.821953491900888
2021-07-31 05:07:17.100934 Epoch 7, Training loss 0.7673381805953468
2021-07-31 05:08:23.241040 Epoch 8, Training loss 0.7284360977313707
2021-07-31 05:09:29.598740 Epoch 9, Training loss 0.7007892883723349
2021-07-31 05:10:35.618412 Epoch 10, Training loss 0.6624118571772295
2021-07-31 05:11:41.712772 Epoch 11, Training loss 0.642104980966929
2021-07-31 05:12:47.957792 Epoch 12, Training loss 0.6146787089460036
2021-07-31 05:13:54.050863 Epoch 13, Training loss 0.5905050756147755
2021-07-31 05:15:00.181673 Epoch 14, Training loss 0.5733622479850374
2021-07-31 05:16:06.356902 Epoch

In [41]:

train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                           shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64,
                                         shuffle=False)

def validate(model, train_loader, val_loader):
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0

        with torch.no_grad():   # we do no need backpropagation for this hence the nograd
            for imgs, labels in loader:
                imgs = imgs.to(device=device)
                labels = labels.to(device=device)
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1) # taking the class at maximum probability as prediction
                total += labels.shape[0]  
                correct += int((predicted == labels).sum())   # calculating the no. of correct predictions.

        print("Accuracy {}: {:.2f}".format(name , correct / total))

validate(model, train_loader, val_loader)

Accuracy train: 0.87
Accuracy val: 0.82


In [42]:
optimizer = optim.SGD(model.parameters(), lr=0.005, weight_decay=0.01)

In [43]:
training_loop(   # 10 iterations with new LR
    n_epochs = 10,                      
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)

2021-07-31 06:45:21.602253 Epoch 1, Training loss 0.22703731442561081
2021-07-31 06:46:26.884151 Epoch 2, Training loss 0.20902931752145443
2021-07-31 06:47:32.521110 Epoch 3, Training loss 0.2060614214385943
2021-07-31 06:48:37.958922 Epoch 4, Training loss 0.2059209347676362
2021-07-31 06:49:43.403156 Epoch 5, Training loss 0.2033590307016202
2021-07-31 06:50:48.684747 Epoch 6, Training loss 0.20368947439333973
2021-07-31 06:51:53.941052 Epoch 7, Training loss 0.20252378454522404
2021-07-31 06:52:59.256680 Epoch 8, Training loss 0.19996077006163498
2021-07-31 06:54:04.548188 Epoch 9, Training loss 0.20431800866904465
2021-07-31 06:55:09.705806 Epoch 10, Training loss 0.20122921518752795


In [44]:
validate(model, train_loader, val_loader)           #now checking acc

Accuracy train: 0.93
Accuracy val: 0.85


In [46]:
optimizer = optim.SGD(model.parameters(), lr=0.001, weight_decay=0.01)     #further lowering LR

In [47]:
training_loop(                    # another 10 iterations
    n_epochs = 10,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)

2021-07-31 06:57:18.192641 Epoch 1, Training loss 0.12092679471034756
2021-07-31 06:58:23.589142 Epoch 2, Training loss 0.09502103105854348
2021-07-31 06:59:28.686982 Epoch 3, Training loss 0.08616054943069587
2021-07-31 07:00:33.795954 Epoch 4, Training loss 0.08041405860963456
2021-07-31 07:01:38.920261 Epoch 5, Training loss 0.07762975376718642
2021-07-31 07:02:44.016825 Epoch 6, Training loss 0.06999396136306855
2021-07-31 07:03:49.534051 Epoch 7, Training loss 0.06919175327179092
2021-07-31 07:04:54.611408 Epoch 8, Training loss 0.06653464321866441
2021-07-31 07:05:59.639167 Epoch 9, Training loss 0.06344245725949688
2021-07-31 07:07:04.652716 Epoch 10, Training loss 0.06189026420011812


In [48]:
validate(model, train_loader, val_loader)           #now checking acc

Accuracy train: 0.98
Accuracy val: 0.88
